In [1]:
v = {'a':1, 'b': 2, 'c': 3} 

In [5]:
def generate_functions():
    values = {k: lambda x:v for k,v in v.items()}
    return values

In [11]:
for k in v.keys():
    print(generate_functions()['a']('something'))

2
2
2


In [18]:
def f():
    a = 5
    lambda_one = lambda x, k=a: k
    a= 10
    return lambda_one

5

In [16]:
l1 = lambda_alone()
l1('something')

NameError: free variable 'a' referenced before assignment in enclosing scope

In [14]:
l2('something')

10